# Fault Environment Example
This tutorial demonstrates how to configure and use a simple BSK-RL environment to model faults in a system with four reaction wheels (RWs).

## Load Modules

In [ ]:
import numpy as np
from typing import Iterable

from Basilisk.architecture import bskLogging
from Basilisk.utilities import macros, orbitalMotion, simIncludeRW
from Basilisk.simulation import reactionWheelStateEffector
from Basilisk.fswAlgorithms import rwNullSpace
from Basilisk.architecture import messaging
from bsk_rl import SatelliteTasking, act, data, obs, scene, sats
from bsk_rl.sim import dyn, fsw
from bsk_rl.utils.orbital import random_orbit, random_unit_vector
from bsk_rl.utils.functional import default_args

bskLogging.setDefaultLogLevel(bskLogging.BSK_WARNING)

## Making Faults Cases
Creating a fault base class and defining individual fault types enables modeling multiple kinds of faults within a single satellite. In this example, a power draw limit is applied to RWs, causing it to operate at reduced speed compared to nominal conditions. By default, while a torque limit is enforced, there are no restrictions on power draw. `time` is used to define the time at which the fault occurs, `reducedLimit` specifies the power draw limit in watts, and `wheel_Idx` indicates which RW is affected by the fault. It can be set to a value from 1 to 4, or to all to apply the fault to every RW.

In [ ]:
class FaultObject:
    def __init__(self, name, time, verbose=True, **kwargs):
        self.name = name
        self.time = time
        self.verbose = verbose
        self.message = None
        self.message_printed = False

    def execute(self, satellite):
        raise NotImplementedError(
            f"{self.name} does not have a custom execute function!"
        )

    def print_message(self, message, satellite):
        if not self.message_printed:
            satellite.logger.info(message)
            self.message_printed = True

    def addFaultToSimulation(self, satellite, listIdx):
        self.uniqueFaultIdx = listIdx  # Index in the faultList array.
        satellite.simulator.createNewEvent(
            f"add{self.name}Fault",
            satellite.dynamics.dyn_rate,
            eventActive=True,
            conditionTime=self.time,
            actionList=[
                f"self.faultList[{self.uniqueFaultIdx}].execute({satellite._satellite_command})",
                f"self.faultList[{self.uniqueFaultIdx}].print({satellite._satellite_command})",
            ],
        )


class RwPowerFault(FaultObject):
    def __init__(self, name, time, reducedLimit, wheelIdx):
        super().__init__(name, time)
        self.reducedLimit = reducedLimit

        if isinstance(wheelIdx, float):
            # int needed around wheelIdx because np.random.choice doesn't return
            # a type int, and the index will not register in execute without it.
            self.wheelIdx = int(wheelIdx)
        elif isinstance(wheelIdx, int) or wheelIdx == "all":
            # option to trigger the fault in all wheels reflecting a larger power issue
            self.wheelIdx = wheelIdx
        else:
            raise ValueError(
                "Fault parameter 'wheelIdx' must either be a number corresponding to a reaction wheel or the string 'all'"
            )

    def execute(self, satellite):
        dynModels = satellite.dynamics
        if self.wheelIdx == 1:
            dynModels.rwFactory.rwList["RW1"].P_max = self.reducedLimit
        elif self.wheelIdx == 2:
            dynModels.rwFactory.rwList["RW2"].P_max = self.reducedLimit
        elif self.wheelIdx == 3:
            dynModels.rwFactory.rwList["RW3"].P_max = self.reducedLimit
        elif self.wheelIdx == 4:
            dynModels.rwFactory.rwList["RW4"].P_max = self.reducedLimit
        elif self.wheelIdx == "all":
            # option to trigger the fault in all wheels (not supported for all fault types)
            dynModels.rwFactory.rwList["RW1"].P_max = self.reducedLimit
            dynModels.rwFactory.rwList["RW2"].P_max = self.reducedLimit
            dynModels.rwFactory.rwList["RW3"].P_max = self.reducedLimit
            dynModels.rwFactory.rwList["RW4"].P_max = self.reducedLimit

    def print(self, satellite):
        if self.wheelIdx == "all":
            self.message = f"RW Power Fault: all RW's power limit reduced to {self.reducedLimit} Watts at {self.time * macros.NANO2MIN} minutes!"
        else:
            self.message = f"RW Power Fault: RW{self.wheelIdx}'s power limit reduced to {self.reducedLimit} Watts at {self.time * macros.NANO2MIN} minutes!"
        super().print_message(self.message, satellite)

## Configure the Simulation Models
* [Dynamics model](../api_reference/sim/dyn.rst): `FullFeaturedDynModel` is used as the base class, and `setup_reaction_wheel_dyn_effector` is overridden to support four RWs. Two additional properties are added: the angle between the Sun and the solar panel, and the speed fraction of each RW.

In [ ]:
class CustomDynModel(dyn.FullFeaturedDynModel):
    @property
    def solar_angle_norm(self) -> float:
        sun_vec_N = (
            self.world.gravFactory.spiceObject.planetStateOutMsgs[self.world.sun_index]
            .read()
            .PositionVector
        )
        sun_vec_N_hat = sun_vec_N / np.linalg.norm(sun_vec_N)
        solar_panel_vec_B = np.array([0, 0, -1])
        mat = np.transpose(self.BN)
        solar_panel_vec_N = np.matmul(mat, solar_panel_vec_B)
        error_angle = np.arccos(np.dot(solar_panel_vec_N, sun_vec_N_hat))

        return error_angle / np.pi

    @property
    def wheel_speeds_frac(self):
        rw_speed = self.wheel_speeds
        return rw_speed[0:4] / (self.maxWheelSpeed * macros.rpm2radsec)

    @default_args(
        wheelSpeeds=lambda: np.random.uniform(-1500, 1500, 4),
        maxWheelSpeed=np.inf,
        u_max=0.200,
    )
    def setup_reaction_wheel_dyn_effector(
        self,
        wheelSpeeds: Iterable[float],
        maxWheelSpeed: float,
        u_max: float,
        priority: int = 997,
        **kwargs,
    ) -> None:
        """Set the RW state effector parameters.

        Args:
            wheelSpeeds: Initial speeds of each wheel [RPM]
            maxWheelSpeed: Failure speed for wheels [RPM]
            u_max: Torque producible by wheel [N*m]
            priority: Model priority.
            kwargs: Ignored
        """

        def balancedHR16Triad(
            useRandom=False, randomBounds=(-400, 400), wheelSpeeds=[500, 500, 500, 500]
        ):
            """Create a set of three HR16 reaction wheels.

            Args:
                useRandom: Use random values for wheel speeds.
                randomBounds: Bounds for random wheel speeds.
                wheelSpeeds: Fixed wheel speeds.

            Returns:
                tuple:
                    * **rwStateEffector**: Reaction wheel state effector instance.
                    * **rwFactory**: Factory containing defined reaction wheels.
                    * **wheelSpeeds**: Wheel speeds.
            """
            rwFactory = simIncludeRW.rwFactory()

            if useRandom:
                wheelSpeeds = np.random.uniform(randomBounds[0], randomBounds[1], 4)
            c = 3 ** (-0.5)
            rwFactory.create(
                "Honeywell_HR16",
                [1, 0, 0],
                maxMomentum=50.0,
                Omega=wheelSpeeds[0],
            )
            rwFactory.create(
                "Honeywell_HR16",
                [0, 1, 0],
                maxMomentum=50.0,
                Omega=wheelSpeeds[1],
            )
            rwFactory.create(
                "Honeywell_HR16",
                [0, 0, 1],
                maxMomentum=50.0,
                Omega=wheelSpeeds[2],
            )
            rwFactory.create(
                "Honeywell_HR16",
                [c, c, c],
                maxMomentum=50.0,
                Omega=wheelSpeeds[3],
            )

            rwStateEffector = reactionWheelStateEffector.ReactionWheelStateEffector()

            return rwStateEffector, rwFactory, wheelSpeeds

        self.maxWheelSpeed = maxWheelSpeed
        self.rwStateEffector, self.rwFactory, _ = balancedHR16Triad(
            useRandom=False,
            wheelSpeeds=wheelSpeeds,
        )
        for RW in self.rwFactory.rwList.values():
            RW.u_max = u_max
        self.rwStateEffector.ModelTag = "ReactionWheels"
        self.rwFactory.addToSpacecraft(
            self.scObject.ModelTag, self.rwStateEffector, self.scObject
        )
        self.simulator.AddModelToTask(
            self.task_name, self.rwStateEffector, ModelPriority=priority
        )

        self.Gs = np.array(
            [
                [1, 0, 0, 1 / np.sqrt(3)],  # RW1 and RW4 x-components
                [0, 1, 0, 1 / np.sqrt(3)],  # RW2 and RW4 y-components
                [0, 0, 1, 1 / np.sqrt(3)],  # RW3 and RW4 z-components
            ]
        )

* [Flight software model](../api_reference/sim/fsw.rst): A custom flight software model is defined to support four RWs. It is based on the `SteeringImagerFSWModel`, with the main modification being the addition of the `rwNullSpace` module. Due to the redundancy of having four RWs, there are infinitely many solutions for mapping the required body control torque to individual RW torques. To address this, once the control torque is computed, the RW null space is used to decelerate the wheels without applying additional torque to the spacecraft.

In [ ]:
class CustomSteeringImagerFSWModel(fsw.SteeringImagerFSWModel):
    def __init__(self, *args, **kwargs) -> None:
        """Convenience type that combines the imaging FSW model with MRP steering for four reaction wheels."""
        super().__init__(*args, **kwargs)

    def _set_config_msgs(self) -> None:
        super()._set_config_msgs()
        self._set_rw_constellation_msg()

    def _set_rw_constellation_msg(self) -> None:
        """Set the reaction wheel constellation message."""
        rwConstellationConfig = messaging.RWConstellationMsgPayload()
        rwConstellationConfig.numRW = self.dynamics.rwFactory.getNumOfDevices()
        rwConfigElementList = []
        for i in range(4):
            rwConfigElementMsg = messaging.RWConfigElementMsgPayload()
            rwConfigElementMsg.gsHat_B = self.dynamics.Gs[:, i]
            rwConfigElementMsg.Js = self.dynamics.rwFactory.rwList[f"RW{i + 1}"].Js
            rwConfigElementMsg.uMax = self.dynamics.rwFactory.rwList[f"RW{i + 1}"].u_max
            rwConfigElementList.append(rwConfigElementMsg)
        rwConstellationConfig.reactionWheels = rwConfigElementList
        self.rwConstellationConfigInMsg = messaging.RWConstellationMsg().write(
            rwConstellationConfig
        )

    def _set_gateway_msgs(self) -> None:
        """Create C-wrapped gateway messages."""
        self.attRefMsg = messaging.AttRefMsg_C()
        self.attGuidMsg = messaging.AttGuidMsg_C()

        self._zero_gateway_msgs()

        # connect gateway FSW effector command msgs with the dynamics
        self.dynamics.rwStateEffector.rwMotorCmdInMsg.subscribeTo(
            self.rwNullSpace.rwMotorTorqueOutMsg
        )
        self.dynamics.thrusterSet.cmdsInMsg.subscribeTo(
            self.thrDump.thrusterOnTimeOutMsg
        )

    class MRPControlTask(fsw.SteeringImagerFSWModel.MRPControlTask):
        def _create_module_data(self) -> None:
            super()._create_module_data()

            self.rwNullSpace = self.fsw.rwNullSpace = rwNullSpace.rwNullSpace()
            self.rwNullSpace.ModelTag = "rwNullSpace"

        def _setup_fsw_objects(self, **kwargs) -> None:
            super()._setup_fsw_objects(**kwargs)
            self.set_rw_null_space(**kwargs)

        @default_args(OmegaGain=0.3)
        def set_rw_null_space(
            self,
            OmegaGain: float,
            **kwargs,
        ) -> None:
            """Define the null space to slow down the wheels."""
            self.rwNullSpace.rwMotorTorqueInMsg.subscribeTo(
                self.rwMotorTorque.rwMotorTorqueOutMsg
            )
            self.rwNullSpace.rwSpeedsInMsg.subscribeTo(
                self.fsw.dynamics.rwStateEffector.rwSpeedOutMsg
            )
            self.rwNullSpace.rwConfigInMsg.subscribeTo(
                self.fsw.rwConstellationConfigInMsg
            )
            self.rwNullSpace.OmegaGain = OmegaGain
            self._add_model_to_task(self.rwNullSpace, priority=1193)

## Configure the Satellite
* [Observations](../api_reference/obs/index.rst): 
    - SatProperties: Body angular velocity, instrument pointing direction, body position, body velocity, battery charge (properties in [flight software model](../api_reference/sim/fsw.rst) or [dynamics model](../api_reference/sim/dyn.rst)). Also, customized dynamics property in CustomDynModel above: Angle between the sun and the solar panel and four RW speed fraction. 
    - OpportunityProperties: Target's priority, normalized location, and target angle (upcoming 32 targets).
    - Time: Simulation time.
    - Eclipse: Next eclipse start and end times. 
* [Actions](../api_reference/act/index.rst):
    - Desat: Manage momentum for the RWs for 60 seconds.
    - Charge: Enter a sun-pointing charging mode for 60 seconds.
    - Image: Image target from upcoming 32 targets

The fault is introduced by overriding the `reset_post_sim_init` function. The probability of the fault occurring can be specified using the `fault_chance` argument, and the time of occurrence can be set using the `fault_time` argument.

In [ ]:
class CustomSatComposed(sats.ImagingSatellite):
    observation_spec = [
        obs.SatProperties(
            dict(prop="omega_BP_P", norm=0.03),
            dict(prop="c_hat_P"),
            dict(prop="r_BN_P", norm=orbitalMotion.REQ_EARTH * 1e3),
            dict(prop="v_BN_P", norm=7616.5),
            dict(prop="battery_charge_fraction"),
            dict(prop="solar_angle_norm"),
            dict(prop="wheel_speeds_frac"),
        ),
        obs.OpportunityProperties(
            dict(prop="priority"),
            dict(prop="r_LP_P", norm=orbitalMotion.REQ_EARTH * 1e3),
            dict(prop="target_angle", norm=np.pi),
            type="target",
            n_ahead_observe=32,
        ),
        obs.Time(),
        obs.Eclipse(norm=5700),
    ]

    action_spec = [
        act.Desat(duration=60.0),
        act.Charge(duration=60.0),
        act.Image(n_ahead_image=32),
    ]

    # Modified the constructor to include the fault chance and list
    def __init__(self, *args, fault_chance=0, fault_time=0.0, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.fault_chance = fault_chance
        self.fault_time = fault_time
        self.faultList = []  # List to store faults

    def reset_post_sim_init(self) -> None:
        super().reset_post_sim_init()

        if np.random.random() < self.fault_chance:
            powerFault = RwPowerFault(
                "rwPowerLimited", self.fault_time, reducedLimit=1.0, wheelIdx=1
            )
            self.faultList = [powerFault]
            self.simulator.faultList = self.faultList
            for i in range(len(self.faultList)):
                self.faultList[i].addFaultToSimulation(self, i)

    dyn_type = CustomDynModel
    fsw_type = CustomSteeringImagerFSWModel

# Configure Satellite Pareameters
When instantiating a satellite, these parameters can be overriden with a constant or 
rerandomized every time the environment is reset using the ``sat_args`` dictionary.

In [ ]:
dataStorageCapacity = 20 * 8e6 * 100
batteryStorageCapacity = 80.0 * 3600 * 2
sat_args = CustomSatComposed.default_sat_args(
    oe=random_orbit,
    imageAttErrorRequirement=0.01,
    imageRateErrorRequirement=0.01,
    batteryStorageCapacity=batteryStorageCapacity,
    storedCharge_Init=lambda: np.random.uniform(0.4, 1.0) * batteryStorageCapacity,
    u_max=0.2,  # More realistic values than 1.0
    K1=0.5,  # Updated value to have smooth and more predictable control
    nHat_B=np.array([0, 0, -1]),
    imageTargetMinimumElevation=np.radians(45),
    rwBasePower=20,
    maxWheelSpeed=1500,
    storageInit=lambda: np.random.randint(
        0 * dataStorageCapacity,
        0.01 * dataStorageCapacity,
    ),
    wheelSpeeds=lambda: np.random.uniform(-900, 900, 4),
    disturbance_vector=lambda: random_unit_vector(),
)

# Make the satellites
satellites = []
satellites.append(
    CustomSatComposed(
        "EO",
        sat_args,
        fault_chance=1.0,
        fault_time=0.0,  # Fault occurs at 0.0 (nano seconds)
    )
)

## Making and interacting the Environment
For this example, the single-agent [SatelliteTasking](../api_reference/index.rst) environment is used. n addition to the configured satellite, the environment requires a [scenario](../api_reference/scene/index.rst), which defines the context in which the satellite operates. In this case, the scenario uses `UniformTargets`, placing 1000 uniformly distributed targets across the Earth’s surface. The environment also takes a [rewarder](../api_reference/data/index.rst), which defines how data collected from the scenario is rewarded. Here, `UniqueImageReward` is used, which assigns rewards based on the sum of the priorities of uniquely imaged targets in each episode.

In [ ]:
env = SatelliteTasking(
    satellite=satellites,
    terminate_on_time_limit=True,
    scenario=scene.UniformTargets(n_targets=1000),
    rewarder=data.UniqueImageReward(),
    sim_rate=0.5,
    max_step_duration=300.0,
    time_limit=95 * 60 * 3,
    log_level="INFO",
    failure_penalty=0,
    # disable_env_checker=True, # For debugging
)

First, the environment is reset. A seed is provided to ensure reproducibility of the results; it can be removed to enable randomized testing.

In [ ]:
observation, info = env.reset(seed=1)

The composed satellite action space returns a human-readable action map and each satellite has the same action space and similar observation space.

In [ ]:
print("Actions:", satellites[0].action_description)
print("States:", env.unwrapped.satellites[0].observation_description, "\n")

# Using the composed satellite features also provides a human-readable state:
for satellite in env.unwrapped.satellites:
    for k, v in satellite.observation_builder.obs_dict().items():
        print(f"{k}:  {v}")

The simulation runs until either the battery is depleted, a RW exceeds its maximum speed (both considered failures), or a timeout occurs (which simply stops the simulation).

In [ ]:
total_reward = 0.0
while True:
    observation, reward, terminated, truncated, info = env.step(
        env.action_space.sample()
    )
    total_reward += reward
    if terminated or truncated:
        print("Episode complete.")
        break

print("Total reward:", total_reward)